In [1]:
## build researcher events, student events, and self-study events
## researcher are simply those with NH citations

In [2]:
import sys

import pandas as pd
import numpy as np
import time
import datetime
import os
from nanoHUB.application import Application
from nanoHUB.configuration import ClusteringConfiguration
from nanoHUB.pipeline.geddes.data import get_default_s3_client
from nanoHUB.dataaccess.lake import S3FileMapper

from nanoHUB.clustering.infra import get_mike_only_clustered_users, get_xufeng_only_clustered_users, get_mike_xufeng_clustered_users
from nanoHUB.clustering.infra import create_clusters_repository, add_cluster_info, get_all_clustered_users, get_all_self_study_users, get_all_self_identified_users

nanoHUB - Serving Students, Researchers & Instructors


In [3]:
pd.set_option('display.max_columns', None)
cwd = os.getcwd()

In [4]:
application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')
wang159_myrmekes_db = application.new_db_engine('rfm_data')

s3_client = get_default_s3_client(application)
derived_data_file_path = 'derived_data_for_users.csv'
processed_mapper = S3FileMapper(s3_client, ClusteringConfiguration().bucket_name_processed)

## Cumulative classroom/research/self-study

In [5]:
today = datetime.datetime.today()
print(today)

2022-02-22 02:20:11.443637


In [6]:
## determine starting year
#if cache_flag == False:
start_year_base = 1999 #2000
#else:
#     print('check the saved file')
#    start_year_base = int(user_breakdown_df['year_month'].to_list()[-1][:4])
    
    

In [7]:
from copy import deepcopy

In [9]:
cache_flag = False
all_months_list = list()
for this_year in range(start_year_base+1,today.year+2):
    if cache_flag == True:
        for this_month in range(1,today.month+1):
            all_months_list.append(datetime.datetime(this_year, this_month, 1))
    else:
        for this_month in range(1,12+1):
            all_months_list.append(datetime.datetime(this_year, this_month, 1))

cache_all_month_list = pd.DataFrame()
cache_all_month_list['Date'] = all_months_list
cache_all_month_list.to_csv(cwd+'/cached_data/all_months_list.csv')
cache_all_month_list.to_csv(cwd+'/cached_data/'+ str(start_year_base)+'_all_months_list.csv')
cache_all_month_list = []
print('allmonths partial list',all_months_list[0:36])


allmonths partial list [datetime.datetime(2000, 1, 1, 0, 0), datetime.datetime(2000, 2, 1, 0, 0), datetime.datetime(2000, 3, 1, 0, 0), datetime.datetime(2000, 4, 1, 0, 0), datetime.datetime(2000, 5, 1, 0, 0), datetime.datetime(2000, 6, 1, 0, 0), datetime.datetime(2000, 7, 1, 0, 0), datetime.datetime(2000, 8, 1, 0, 0), datetime.datetime(2000, 9, 1, 0, 0), datetime.datetime(2000, 10, 1, 0, 0), datetime.datetime(2000, 11, 1, 0, 0), datetime.datetime(2000, 12, 1, 0, 0), datetime.datetime(2001, 1, 1, 0, 0), datetime.datetime(2001, 2, 1, 0, 0), datetime.datetime(2001, 3, 1, 0, 0), datetime.datetime(2001, 4, 1, 0, 0), datetime.datetime(2001, 5, 1, 0, 0), datetime.datetime(2001, 6, 1, 0, 0), datetime.datetime(2001, 7, 1, 0, 0), datetime.datetime(2001, 8, 1, 0, 0), datetime.datetime(2001, 9, 1, 0, 0), datetime.datetime(2001, 10, 1, 0, 0), datetime.datetime(2001, 11, 1, 0, 0), datetime.datetime(2001, 12, 1, 0, 0), datetime.datetime(2002, 1, 1, 0, 0), datetime.datetime(2002, 2, 1, 0, 0), datetime

In [10]:
# clustered_NH_username2;researcher_NH_username2;self_NH_username2
clustered_track = []
clustered_m_track = []
clustered_x_track = []
clustered_mx_track = []
self_track = []

# clustered_track_per = []
# self_track_per = []
# researcher_track_per = []

In [26]:
clustered_track_df = pd.read_csv(cwd+'/cached_data/clustered_track.csv',usecols=['track'])
#display(clustered_track_df)

self_identified_track_df = pd.read_csv(cwd+'/cached_data/self_identified_track.csv',usecols=['track'])
#display(self_identified_track_df)
#print('self_identified_track_df[0:30]',self_identified_track_df[0:30])
self_identified_track = self_identified_track_df['track'].tolist()
print('self_identified_track[0:30]',self_identified_track[0:30])
print('len(self_identified_track)',len(self_identified_track))

researcher_track_df = pd.read_csv(cwd+'/cached_data/researcher_track.csv',usecols=['track'])
researcher_track = researcher_track_df['track'].tolist()
print('researcher_track[0:30]',researcher_track[0:30])
print('len(researcher_track)',len(researcher_track))

self2_track_df = pd.read_csv(cwd+'/cached_data/self2_track.csv',usecols=['track'])
self2_track = self2_track_df['track'].tolist()
print('self2_track[0:30]',self2_track[0:30])
print('len(self2_track)',len(self2_track))

simtotal_track_df = pd.read_csv(cwd+'/cached_data/simtotal_track.csv',usecols=['track'])
simtotal_track = simtotal_track_df['track'].tolist()
print('simtotal_track[0:30]',simtotal_track[0:30])
print('len(simtotal_track)',len(simtotal_track))

#The clustering data needs to be stiched together
clustered_track_df = pd.read_csv(cwd+'/cached_data/2016_clustered_track.csv',usecols=['track'])
clustered_track = clustered_track_df['track'].tolist()
print('clustered_track[0:30]',clustered_track[0:30])
print('len(clustered_track)',len(clustered_track))





self_identified_track[0:30] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(self_identified_track) 288
researcher_track[0:30] [0, 0, 8, 11, 13, 14, 15, 17, 21, 24, 24, 25, 27, 33, 35, 36, 39, 39, 36, 35, 37, 37, 39, 40, 39, 37, 35, 34, 31, 37]
len(researcher_track) 288
self2_track[0:30] [0, 0, 45, 55, 68, 74, 76, 92, 184, 235, 293, 343, 372, 571, 707, 731, 756, 764, 774, 765, 765, 771, 730, 696, 640, 622, 622, 608, 359, 347]
len(self2_track) 288
simtotal_track[0:30] [0, 0, 133, 199, 244, 263, 275, 293, 389, 443, 501, 554, 588, 797, 920, 927, 860, 846, 844, 832, 833, 841, 800, 762, 700, 678, 676, 660, 405, 428]
len(simtotal_track) 288
clustered_track[0:30] [0, 0, 6, 7, 10, 11, 13, 13, 15, 16, 16, 16, 17, 21, 20, 21, 18, 17, 15, 14, 12, 13, 13, 13, 13, 13, 12, 12, 11, 15]
len(clustered_track) 204


In [33]:

    cache_clustered_track = pd.DataFrame()
    cache_clustered_track['track'] = clustered_track
    cache_clustered_track.to_csv(cwd+'/cached_data/clustered_track.csv')
    cache_clustered_track = []
    
    cache_clustered_m_track = pd.DataFrame()
    cache_clustered_m_track['track'] = clustered_m_track
    cache_clustered_m_track.to_csv(cwd+'/cached_data/clustered_m_track.csv')
    cache_clustered_m_track = []

    cache_clustered_x_track = pd.DataFrame()
    cache_clustered_x_track['track'] = clustered_x_track
    cache_clustered_x_track.to_csv(cwd+'/cached_data/clustered_x_track.csv')
    cache_clustered_x_track = []

    cache_clustered_mx_track = pd.DataFrame()
    cache_clustered_mx_track['track'] = clustered_mx_track
    cache_clustered_mx_track.to_csv(cwd+'/cached_data/clustered_mx_track.csv')
    cache_clustered_mx_track = []

    cache_self_track = pd.DataFrame()
    cache_self_track['track'] = self_track
    cache_self_track.to_csv(cwd+'/cached_data/self_track.csv')
    cache_self_track = []

    

NameError: name 'deepcopy' is not defined

In [ ]:
print(len(clustered_track))
print(len(clustered_m_track))
print(len(clustered_x_track))
print(len(clustered_mx_track))
print(len(all_months_list))
print(len(researcher_track))

In [ ]:
#print(user_breakdown_df2.head())

In [ ]:

if cache_flag == True:
    user_breakdown_df2 = pd.DataFrame()
    user_breakdown_df2['year_month'] = all_months_list
    user_breakdown_df2['clustered_track'] = clustered_track
    user_breakdown_df2['clustered_m_track'] = clustered_m_track
    user_breakdown_df2['clustered_x_track'] = clustered_x_track
    user_breakdown_df2['clustered_mx_track'] = clustered_mx_track
    user_breakdown_df2['self_identified_track'] = self_identified_track
    user_breakdown_df2['self_track'] = self_track
    user_breakdown_df2['researcher_track'] = researcher_track
    
    user_breakdown_df = pd.concat([user_breakdown_df, user_breakdown_df2], ignore_index=True)
    display(user_breakdown_df.head(2))
else:
    user_breakdown_df = pd.DataFrame()
    if today.month == 12:
        user_breakdown_df['year_month'] = all_months_list #months_bin['year_month'].to_list()[13:]
        user_breakdown_df['clustered_track'] = clustered_track#[:len(all_months_list)]
        user_breakdown_df['clustered_m_track'] = clustered_m_track#[:len(all_months_list)]
        user_breakdown_df['clustered_x_track'] = clustered_x_track#[:len(all_months_list)]
        user_breakdown_df['clustered_mx_track'] = clustered_mx_track#[:len(all_months_list)]
        user_breakdown_df['self_identified_track'] = self_identified_track
        user_breakdown_df['self_track'] = self_track#[:len(all_months_list)]
        user_breakdown_df['researcher_track'] = researcher_track#[:len(all_months_list)]        
    else:
        user_breakdown_df['year_month'] = all_months_list #-(12-today.month)
        user_breakdown_df['clustered_track'] = clustered_track#[:len(all_months_list[:])]
        user_breakdown_df['clustered_m_track'] = clustered_m_track#[:len(all_months_list[:])]
        user_breakdown_df['clustered_x_track'] = clustered_x_track#[:len(all_months_list[:])]
        user_breakdown_df['clustered_mx_track'] = clustered_mx_track#[:len(all_months_list[:])]
        user_breakdown_df['self_identified_track'] = self_identified_track
        user_breakdown_df['self_track'] = self_track#[:len(all_months_list[:])]
        user_breakdown_df['researcher_track'] = researcher_track#[:len(all_months_list[:])]
    display(user_breakdown_df.head(2))

In [ ]:
# user_breakdown_df.to_csv(cwd+'/cached_data/temp.csv')

In [ ]:
import re

In [ ]:
## read the dev_flags.txt to determine where to save
with open(cwd+'/dev_flags.txt','r') as f:
    raw_flags = f.read()
flag_pattern = re.compile(r'= \w+')
flags1 = re.findall(flag_pattern,raw_flags)    
flags_result = [i[2:] for i in flags1]
dev_flag = flags_result[0]
prod_flag = flags_result[1]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
sns.set_style("whitegrid")

plt.figure(figsize=(9,6))
# user_breakdown_df2 = user_breakdown_df.iloc[:-(12-today.month),:]
user_breakdown_df2 = user_breakdown_df.iloc[:-12-(12-today.month),:]
plt.stackplot(user_breakdown_df2.year_month, \
#               user_breakdown_df2[['clustered_track','self_track']].to_numpy().T, \
#              labels=['Classroom', 'Unclassified'], \
#              colors=['green', 'orange']); #Self-study              
              user_breakdown_df2[['clustered_m_track', 'clustered_mx_track', 'clustered_x_track', 'researcher_track','self_identified_track','self_track']].to_numpy().T, \
             labels=['Classroom M','Classroom MX','Classroom X','Research', 'Researcher','Unclassified'], \
             colors=['lawngreen','green', 'yellowgreen','red', 'fuchsia','orange']); #Self-study

plt.legend(loc='upper left')#,fontsize=14);
plt.ylabel('12-month Trailing Total')#,fontsize=15)
plt.xlim([datetime.datetime(2001,1,1), datetime.datetime(2021,7,1)])#,fontsize=15)
# plt.show()

if dev_flag == 'True':
    plt.savefig(cwd+'/plots_local/300_simusers_class_res_unclass_12months_GK.eps', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/300_simusers_class_res_unclass_12months_GK.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/300_simusers_class_res_unclass_12months_GK.png', dpi=300, bbox_inches='tight')
elif prod_flag == 'True':
    plt.savefig(cwd+'/plots_production/300_simusers_class_res_unclass_12months_GK.eps', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/300_simusers_class_res_unclass_12months_GK.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/300_simusers_class_res_unclass_12months_GK.png', dpi=300, bbox_inches='tight')
else:
    print('no printing flag is active')

In [ ]:
total = user_breakdown_df2['clustered_track'].to_numpy()+user_breakdown_df2['researcher_track'].to_numpy()+\
                        user_breakdown_df2['self_track'].to_numpy() + user_breakdown_df2['self_identified_track'].to_numpy()
display(len(total))
user_breakdown_df2['clustered_track_per'] = 100*np.divide(user_breakdown_df2['clustered_track'],total)
user_breakdown_df2['self_identified_track_per'] = 100*np.divide(user_breakdown_df2['self_identified_track'], total)
user_breakdown_df2['researcher_track_per'] = 100*np.divide(user_breakdown_df2['researcher_track'], total)
user_breakdown_df2['self_track_per'] = 100*np.divide(user_breakdown_df2['self_track'], total)

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(user_breakdown_df2.year_month, user_breakdown_df2.clustered_track_per, label='classroom', \
             color='green');
plt.plot(user_breakdown_df2.year_month, user_breakdown_df2.researcher_track_per, label='research', \
             color='red');
plt.plot(user_breakdown_df2.year_month, user_breakdown_df2.self_identified_track_per, label='self-identified', \
             color='magenta');
plt.plot(user_breakdown_df2.year_month, user_breakdown_df2.self_track_per, label='self-study', \
             color='orange');

plt.ylim([0, 100])
plt.legend(loc='upper right');
plt.ylabel('Percentage (%)');
plt.xlim([datetime.datetime(2001,1,1), datetime.datetime(2021,7,1)])
# plt.show()

if dev_flag == 'True':
    plt.savefig(cwd+'/plots_local/300_simusers_class_res_unclass_12months_percent_copied.eps', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/300_simusers_class_res_unclass_12months_percent_copied.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/300_simusers_class_res_unclass_12months_percent_copied.png', dpi=300, bbox_inches='tight')
elif prod_flag == 'True':
    plt.savefig(cwd+'/plots_production/300_simusers_class_res_unclass_12months_percent_copied.eps', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/300_simusers_class_res_unclass_12months_percent_copied.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/300_simusers_class_res_unclass_12months_percent_copied.png', dpi=300, bbox_inches='tight')
else:
    print('no printing flag is active')

In [ ]:
user_breakdown_df2.to_csv(cwd+'/cached_data/nh_user_breakdown.csv')